# Real Estate Location Mapping: Finding Nearest Amenities

This notebook demonstrates how to enrich real estate property data by identifying the nearest metro stations, schools, and hospitals for each property. We'll use geographical distance calculations to link properties to these amenities and save the enriched data to an Excel file.

## 1. Setting Up the Environment and Loading Data

First, we import the necessary libraries. If you don't have them installed, you can install them using `pip`:
`pip install pandas geopy openpyxl`

Then, we load the main real estate dataset and separate datasets for metro stations, schools, and hospitals. Ensure all these Excel files are in the same directory as this notebook.

In [ ]:
import pandas as pd
from geopy.distance import geodesic

print("Loading data files...")
# Load real estate data (ensure this file exists and has 'X' and 'Y' for coordinates)
real_estate_df = pd.read_excel("Delhi_RealEstate_Data.xlsx")
# Load amenity data (ensure these files exist and have 'X', 'Y', and their respective IDs)
metro_df = pd.read_excel("MetroStations_Data.xlsx")
school_df = pd.read_excel("Schools_Data.xlsx")
hospital_df = pd.read_excel("Hospitals_Data.xlsx")
print(f"Loaded {len(real_estate_df)} real estate properties")

## 2. Defining the Nearest ID Function

This function calculates the geodesic distance (shortest distance over the earth's surface) between a source location (a real estate property) and all target locations (amenities). It then returns the IDs of the `k` nearest amenities.

In [ ]:
def find_nearest_ids(source_row, target_df, target_id_col, k):
    # Extract coordinates from the source row (real estate property)
    source_coords = (source_row["Y"], source_row["X"])
    
    # Calculate geodesic distance from source to all target locations
    target_df["distance"] = target_df.apply(
        lambda row: geodesic(source_coords, (row["Y"], row["X"])).meters,
        axis=1
    )
    
    # Get the IDs of the k nearest target locations based on distance
    nearest_ids = target_df.nsmallest(k, "distance")[target_id_col].tolist()
    return nearest_ids

## 3. Mapping Nearest Amenities to Real Estate Properties

We apply the `find_nearest_ids` function to each real estate property in the DataFrame to find the nearest metro stations, schools, and hospitals. The results are stored in new columns in the `real_estate_df`.

In [ ]:
print("Finding nearest metro stations...")
# Find the 2 nearest metro stations for each property
real_estate_df[["nearest_metro_1", "nearest_metro_2"]] = real_estate_df.apply(
    lambda row: pd.Series(find_nearest_ids(row, metro_df, "m_id", 2)), axis=1
)
print("Metro stations mapped")

print("Finding nearest schools...")
# Find the 3 nearest schools for each property
real_estate_df[["nearest_school_1", "nearest_school_2", "nearest_school_3"]] = real_estate_df.apply(
    lambda row: pd.Series(find_nearest_ids(row, school_df, "s_id", 3)), axis=1
)
print("Schools mapped")

print("Finding nearest hospitals...")
# Find the 3 nearest hospitals for each property
real_estate_df[["nearest_hospital_1", "nearest_hospital_2", "nearest_hospital_3"]] = real_estate_df.apply(
    lambda row: pd.Series(find_nearest_ids(row, hospital_df, "h_id", 3)), axis=1
)
print("Hospitals mapped")

## 4. Saving Enriched Data

Finally, the DataFrame with the newly added nearest amenity IDs is saved to a new Excel file. This enriched dataset can then be used for further analysis or modeling.

In [ ]:
print("Saving enriched data...")
real_estate_df.to_excel("Delhi_RealEstate_LocationMaped.xlsx", index=False)
print("Enriched file saved as Delhi_RealEstate_LocationMaped.xlsx")